In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET

# (1) 코스피 상장법인 목록 불러오기
df_kospi = pd.read_csv("data\상장법인목록.csv", encoding="cp949")[["회사명", "종목코드"]]
df_kospi.columns = ["corp_name", "stock_code"]
df_kospi["stock_code"] = df_kospi["stock_code"].astype(str).str.zfill(6)

# (2) CORPCODE.xml 직접 경로에서 파싱
tree = ET.parse("data\corpcode_data\CORPCODE.xml")
root = tree.getroot()

# (3) corp_code 정보 추출
corp_info = []
for child in root:
    corp_code = child.findtext("corp_code")
    corp_name = child.findtext("corp_name")
    stock_code = child.findtext("stock_code")
    corp_info.append({"corp_name": corp_name, "corp_code": corp_code, "stock_code": stock_code})

df_corp_code = pd.DataFrame(corp_info)

# (4) 코스피 기업과 corp_code 매핑
df_merged = pd.merge(df_kospi, df_corp_code, on="stock_code", how="inner")
df_merged = df_merged[["corp_name_x", "stock_code", "corp_code"]]
df_merged.columns = ["corp_name", "stock_code", "corp_code"]

# 확인
df_merged

<>:10: SyntaxWarning: invalid escape sequence '\c'
<>:10: SyntaxWarning: invalid escape sequence '\c'
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_7432\3706037674.py:10: SyntaxWarning: invalid escape sequence '\c'
  tree = ET.parse("data\corpcode_data\CORPCODE.xml")


,corp_name,stock_code,corp_code
0,한화비전,489790,01867758
1,HD현대마린솔루션,443060,01194689
2,에이피알,278470,01190568
3,엘앤에프,066970,00398701
4,포스코DX,022100,00155212
...,...,...,...
195,대한전선,001440,00113207
196,한국앤컴퍼니,000240,00160047
197,대한항공,003490,00113526
198,유한양행,000100,00145109


In [21]:
import yfinance as yf
import pandas as pd
import time

# 1. stock_code를 yfinance용 ticker로 변환
def to_yf_ticker(code):
    code_str = str(code).zfill(6)  # 앞에 0 채우기
    return f"{code_str}.KS"

# 2. 예시: df_merged에서 stock_code만 추출
tickers = df_merged['stock_code'].apply(to_yf_ticker).tolist()

# 3. 데이터프레임 초기화
price_data = pd.DataFrame()

# 4. 각 종목의 종가 불러오기 (최근 1개월 예시)
for ticker in tickers:
    try:
        df = yf.download(ticker, start="2024-06-01", end="2024-06-26")['Close']
        df.name = ticker
        price_data = pd.concat([price_data, df], axis=1)
        time.sleep(0.5)  # 너무 빠르게 요청 시 서버 차단 방지
    except Exception as e:
        print(f"{ticker} 다운로드 실패: {e}")

# 5. 결과 확인
print(price_data.tail())

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['489790.KS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-06-01 -> 2024-06-26) (Yahoo error = "Data doesn\'t exist for startDate = 1717167600, endDate = 1719327600")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100

Ticker      489790.KS      443060.KS  278470.KS  066970.KS     022100.KS  \
Date                                                                       
2024-06-19        NaN  139005.203125    75200.0   153200.0  40060.500000   
2024-06-20        NaN  139393.734375    72900.0   153400.0  39562.851562   
2024-06-21        NaN  134051.109375    79800.0   152000.0  39065.207031   
2024-06-24        NaN  134731.093750    80000.0   146700.0  38268.972656   
2024-06-25        NaN  134731.093750    75800.0   144600.0  37622.035156   

Ticker      450080.KS  454910.KS  457190.KS  456040.KS  373220.KS  ...  \
Date                                                               ...   
2024-06-19   108300.0    78700.0    52600.0   107500.0   343000.0  ...   
2024-06-20   102800.0    77300.0    48950.0   106000.0   340000.0  ...   
2024-06-21    98100.0    81700.0    48900.0   103300.0   333000.0  ...   
2024-06-24    95200.0    79300.0    47600.0   100200.0   331000.0  ...   
2024-06-25    91100.0  

In [23]:
# 1. 티커 변환 함수
def to_yf_ticker(stock_code):
    return f"{str(stock_code).zfill(6)}.KS"

# 2. 시작/종료 날짜 설정 (10년치)
end_date = pd.Timestamp.today()
start_date = end_date - pd.DateOffset(years=10)

# 3. 결과 저장용 데이터프레임 초기화
price_data = pd.DataFrame()

# 4. 변동성 결과 저장용 컬럼 초기화
df_merged['vol_1y'] = np.nan
df_merged['vol_3y'] = np.nan
df_merged['vol_5y'] = np.nan
df_merged['vol_10y'] = np.nan

# 5. 오류 종목 저장 리스트
failed_tickers = []

# 6. 종목 루프
for idx, row in df_merged.iterrows():
    corp_name = row['corp_name']
    stock_code = row['stock_code']
    ticker = to_yf_ticker(stock_code)

    try:
        df = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'), progress=False)[['Close']]
        df.rename(columns={'Close': corp_name}, inplace=True)

        df.fillna(method='ffill', inplace=True)
        df.fillna(method='bfill', inplace=True)

        log_ret = np.log(df / df.shift(1)).dropna()

        for years, colname in zip([1, 3, 5, 10], ['vol_1y', 'vol_3y', 'vol_5y', 'vol_10y']):
            start_cutoff = end_date - pd.DateOffset(years=years)
            log_ret_subset = log_ret[log_ret.index >= start_cutoff]

            if not log_ret_subset.empty:
                vol = log_ret_subset.std().values[0] * np.sqrt(252)
                df_merged.at[idx, colname] = vol

        time.sleep(0.3)  # 딜레이 유지

    except Exception as e:
        failed_tickers.append((corp_name, ticker))

# 7. 결과 요약 출력
print("✅ 변동성 계산 완료")
print(f"총 처리 종목 수: {len(df_merged)}")
print(f"실패한 종목 수: {len(failed_tickers)}")

if failed_tickers:
    print("\n❌ 실패한 종목 목록:")
    for name, ticker in failed_tickers:
        print(f" - {name} ({ticker})")

# 8. 결과 미리보기
print("\n📈 변동성 결과 샘플:")
print(df_merged[['corp_name', 'vol_1y', 'vol_3y', 'vol_5y', 'vol_10y']].head())

C:\Users\kiwoom\AppData\Local\Temp\ipykernel_7432\4224691285.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_7432\4224691285.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_7432\4224691285.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_7432\4224691285.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
C:\Users\kiwoom\

✅ 변동성 계산 완료
총 처리 종목 수: 200
실패한 종목 수: 0

📈 변동성 결과 샘플:
   corp_name    vol_1y    vol_3y    vol_5y   vol_10y
0       한화비전  0.853230  0.853230  0.853230  0.853230
1  HD현대마린솔루션  0.583690  0.641923  0.641923  0.641923
2       에이피알  0.643910  0.662425  0.662425  0.662425
3       엘앤에프  0.700304  0.654724  0.638332  0.600102
4      포스코DX  0.615264  0.688626  0.589515  0.488378


In [11]:
import pandas as pd
import yfinance as yf
import numpy as np
import time

# 2. yfinance 티커 변환 함수 (코스피는 '.KS' 붙임)
def to_yf_ticker(stock_code):
    return f"{stock_code}.KS"

# 3. 각 기업 종가를 저장할 빈 데이터프레임 선언 (인덱스는 날짜)
price_data = pd.DataFrame()

# 4. yfinance에서 1년치 종가 불러오기 (1년 전 오늘부터 오늘까지)
start_date = pd.Timestamp.today() - pd.Timedelta(days=365)
end_date = pd.Timestamp.today()

for idx, row in df_merged.iterrows():
    corp_name = row['corp_name']
    stock_code = row['stock_code']
    ticker = to_yf_ticker(stock_code)

    try:
        df = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'), progress=False)[['Close']]
        df.rename(columns={'Close': corp_name}, inplace=True)

        # 첫 데이터일 때 price_data가 비어있으면 바로 할당
        if price_data.empty:
            price_data = df
        else:
            price_data = price_data.join(df, how='outer')

        print(f"{corp_name} ({ticker}) 데이터 불러오기 성공")
        time.sleep(0.3)  # 과부하 방지용 딜레이
    except Exception as e:
        print(f"{corp_name} ({ticker}) 데이터 불러오기 실패: {e}")

# 5. 종가 데이터 결측치 처리 (앞선 날짜로 채우기, 없으면 뒤에서 채우기)
price_data.fillna(method='ffill', inplace=True)
price_data.fillna(method='bfill', inplace=True)

# 6. 로그수익률 계산
log_returns = np.log(price_data / price_data.shift(1))

# 7. NaN 제거 (첫 행 NaN 제거)
log_returns.dropna(how='all', inplace=True)

# 8. 동일 가중치 (모든 주식에 동일 비중)
weights = np.repeat(1 / log_returns.shape[1], log_returns.shape[1])

# 9. 가중평균 시장 로그수익률 계산
market_log_return = log_returns.dot(weights)

# 10. 연 환산 변동성 계산 (일별 표준편차 × sqrt(252))
market_volatility = market_log_return.std() * np.sqrt(252)

print(f"시장 변동성: {market_volatility*100:.2f}%")

한화비전 (489790.KS) 데이터 불러오기 성공
HD현대마린솔루션 (443060.KS) 데이터 불러오기 성공
에이피알 (278470.KS) 데이터 불러오기 성공
엘앤에프 (066970.KS) 데이터 불러오기 성공
포스코DX (022100.KS) 데이터 불러오기 성공
에코프로머티 (450080.KS) 데이터 불러오기 성공
두산로보틱스 (454910.KS) 데이터 불러오기 성공
이수스페셜티케미컬 (457190.KS) 데이터 불러오기 성공
OCI (456040.KS) 데이터 불러오기 성공
LG에너지솔루션 (373220.KS) 데이터 불러오기 성공
SK스퀘어 (402340.KS) 데이터 불러오기 성공
카카오페이 (377300.KS) 데이터 불러오기 성공
HD현대중공업 (329180.KS) 데이터 불러오기 성공
크래프톤 (259960.KS) 데이터 불러오기 성공
카카오뱅크 (323410.KS) 데이터 불러오기 성공
에스디바이오센서 (137310.KS) 데이터 불러오기 성공
F&F (383220.KS) 데이터 불러오기 성공
SK아이이테크놀로지 (361610.KS) 데이터 불러오기 성공
SK바이오사이언스 (302440.KS) 데이터 불러오기 성공
DL이앤씨 (375500.KS) 데이터 불러오기 성공
하이브 (352820.KS) 데이터 불러오기 성공
에스케이바이오팜 (326030.KS) 데이터 불러오기 성공
한화시스템 (272210.KS) 데이터 불러오기 성공
포스코퓨처엠 (003670.KS) 데이터 불러오기 성공
더블유게임즈 (192080.KS) 데이터 불러오기 성공
우리금융지주 (316140.KS) 데이터 불러오기 성공
한일시멘트 (300720.KS) 데이터 불러오기 성공
HS효성첨단소재 (298050.KS) 데이터 불러오기 성공
효성중공업 (298040.KS) 데이터 불러오기 성공
효성티앤씨 (298020.KS) 데이터 불러오기 성공
셀트리온 (068270.KS) 데이터 불러오기 성공
SK케미칼 (285130.KS) 데이터 불러오기 성공
BGF리테일 (282330.

C:\Users\kiwoom\AppData\Local\Temp\ipykernel_7432\155143035.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_data.fillna(method='ffill', inplace=True)
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_7432\155143035.py:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_data.fillna(method='bfill', inplace=True)


## --> df_merged로 기업별로 종가 구해서 시장 변동성 구함

In [13]:
def get_index_volatility(ticker, years):
    """
    ticker : yfinance 지수 티커 (예: '^KS200')
    years : 변동성 계산 기간 (정수, 년 단위)
    """
    end_date = pd.Timestamp.today()
    start_date = end_date - pd.Timedelta(days=365 * years)
    
    # 지수 가격 데이터 다운로드
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), 
                      end=end_date.strftime('%Y-%m-%d'), progress=False)
    
    if data.empty:
        print(f"{ticker} 데이터가 없습니다: 기간 {years}년")
        return np.nan
    
    # 로그수익률 계산
    log_returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()
    
    # 혹시 DataFrame 형태면 Series로 변환 (보통은 Series임)
    if isinstance(log_returns, pd.DataFrame):
        log_returns = log_returns.iloc[:, 0]
    
    # 연 환산 변동성 계산 (일별 표준편차 * sqrt(252))
    volatility = log_returns.std() * np.sqrt(252)
    return volatility

ks200_ticker = '^KS200'

for years in [1, 3, 5, 10]:
    vol = get_index_volatility(ks200_ticker, years)
    if pd.isna(vol):
        print(f"{years}년 데이터 없음")
    else:
        print(f"{years}년 코스피200 지수 변동성: {vol*100:.2f}%")

1년 코스피200 지수 변동성: 23.48%
3년 코스피200 지수 변동성: 19.46%
5년 코스피200 지수 변동성: 18.98%
10년 코스피200 지수 변동성: 18.08%


## --> yfinance로 코스피200 불러와서 지수 변동성 구함